# Part 8. 파생 (컬럼)피처 생성 (전처리의 목적 지점)
이 장은 전처리의 방향이 바뀌는 지점이다. <br>
지금까지는 데이터를 "정상화"했다면, 이제는 분석에 필요한 의미를 새로 만들어내는 단계다.

## 8.1 파생 피처란 무엇인가
파생 피처(derived feature)란 기존 컬럼을 가공해서 새로 만든 컬럼이다. <br>
이 컬럼은 원본 데이터에는 없지만 분석 질문에 직접적으로 필요한 정보다

## 8.2 파생 피처는 언제 필요한가?
원본 데이터만으로 다음 상황에서 파생 피처가 필요하다.
- 원본 값 그대로는 비교 기준이 애매할 때
- 그룹 기준을 만들기 어려울 때
- 집계 질문을 바로 던질 수 없을 때

이럴때엔
- 비교 기준
- 그룹 기준
- 집계 기준

즉, "이 컬럼으로 바로 질문을 던질 수 있는가?"에 망설임이 생기면 파생 피처를 만들 시점이다.


### 기억해야할 파생 피처 포인트
파생 피처에 사용할 피처들은 기본적으로 분석 가능한 dtype 상태여야 한다.<br>
즉, 값의 정제가 어느정도 진행된 뒤에 해야하고 항상 파생 피처의 생성은 후순위 이다.

In [3]:
import pandas as pd

df = pd.DataFrame({
    "order_id": [1, 2, 3, 4, 5],
    "order_date": [ "2026-01-01", "2026-01-01", "2026-01-02", "2026-02-01", "2026-02-03"],
    "menu": ["Latte", "Americano", "Latte", "Mocha", "Latte"],
    "price": [5000, 4500, 5000, 5500, 5000],
    "qty": [1, 2, 1, 1, 3]
})

df["order_date"] = pd.to_datetime(df["order_date"]) #미리 데이트타임으로 형변환

print("원본 DataFrame")
print(df)

print("\nDataFrame 정보")
print(df.info())

원본 DataFrame
   order_id order_date       menu  price  qty
0         1 2026-01-01      Latte   5000    1
1         2 2026-01-01  Americano   4500    2
2         3 2026-01-02      Latte   5000    1
3         4 2026-02-01      Mocha   5500    1
4         5 2026-02-03      Latte   5000    3

DataFrame 정보
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   order_id    5 non-null      int64         
 1   order_date  5 non-null      datetime64[ns]
 2   menu        5 non-null      object        
 3   price       5 non-null      int64         
 4   qty         5 non-null      int64         
dtypes: datetime64[ns](1), int64(3), object(1)
memory usage: 328.0+ bytes
None


### 총 금액 파생 피처
현 데이터프레임으론 주문당 매출 이라는 질문에 대답할 수 없다.<br>
때문에 price와 qty 컬럼을 사용해 파생피처 sales를 만들고자 한다.

In [4]:
df["sales"] = df["price"] * df["qty"]

print(df)
print("\n")
print(df.info())

   order_id order_date       menu  price  qty  sales
0         1 2026-01-01      Latte   5000    1   5000
1         2 2026-01-01  Americano   4500    2   9000
2         3 2026-01-02      Latte   5000    1   5000
3         4 2026-02-01      Mocha   5500    1   5500
4         5 2026-02-03      Latte   5000    3  15000


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   order_id    5 non-null      int64         
 1   order_date  5 non-null      datetime64[ns]
 2   menu        5 non-null      object        
 3   price       5 non-null      int64         
 4   qty         5 non-null      int64         
 5   sales       5 non-null      int64         
dtypes: datetime64[ns](1), int64(4), object(1)
memory usage: 368.0+ bytes
None


## 8.3) 날짜 파생 피처의 대표 예
날짜 데이터는 파생 피처를 만들기 가장 좋은 대상이다. <br>
대표적인 날짜 파생 피처는 다음과 같다.
- 연(year)
- 월(month)
- 요일(day of week)

이 파생 피처들은
- 시간 흐름 비교
- 주기성 분석
- 그룹화

의 기본 재료가 된다.

날짜 컬럼이 있다면 "그냥 두는 것"이 아니라 분해해서 쓰는 것이 기본이다.


### 주의사항
문자열 날짜 상태에서는 시간 분석을 할 수 없다.<br>
때문에 to_datetime로 미리 문자열을 datetime으로 바꾸자

### 사용가능한 datetime 메소드
문장 구조
```python
datetime_타입_Series.dt.속성 # datetime 접근자 (문자열 접근자 .str 생각하기)

#또는

datetime_타입_Series.dt.메소드()
``` 
- 주의사항
    - 반드시 Series여야 한다
- 속성엔 다음과 같은 값이 들어올수 있다.
    - year
    - month
    - day
    - hour 등...
- 메소드엔 다양한 키워드가 들어갈수 있다.

In [5]:
df["year"] = df["order_date"].dt.year
df["month"] = df["order_date"].dt.month
df["weekday"] = df["order_date"].dt.day_name()

df[["order_date", "year", "month", "weekday"]]

,order_date,year,month,weekday
0,2026-01-01,2026,1,Thursday
1,2026-01-01,2026,1,Thursday
2,2026-01-02,2026,1,Friday
3,2026-02-01,2026,2,Sunday
4,2026-02-03,2026,2,Tuesday


## 8.4) 파생 피처의 목적: 분석 질문을 바꾸는 힘
파생 피처의 진짜 목적은 컬럼을 늘리는 것이 아니다.
> 분석 질문을 단순하게 만드는 것이다.

파생 피처가 잘 만들어지면 분석 코드는 단순해지고, 질문은 더 구체적으로 변한다.
- "날짜별 매출" → "월별 매출"
- "개별 주문" → "메뉴별 평균 가격"
- "원본 로그" → "비교 가능한 그룹"

## 8.5) 파생 피처 생성 시 주의할 점
파생 피처는 많이 만든다고 좋은 게 아니다. 다음 기준을 항상 함께 생각해야 한다.
1. 원본 컬럼을 훼손하지 않는다<br>
→ 새 컬럼으로 만든다
2. 의미가 명확한 이름을 쓴다<br>
→ 분석 기준이 바로 보이도록
3. 필요 이상으로 만들지 않는다<br>
→ 질문이 생길 때 만든다

파생 피처는 많을수록 좋은 게 아니라 목적에 맞을수록 좋다